## memory 공유 확인 위한 dummy chain 생성

In [129]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [130]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [131]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [132]:
dummy_chain = dummy_prompt | model 

In [133]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [134]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 5b6f3689-b45b-445b-965a-94ac14dcffe0 not found for run dcde2ba9-4a28-4e19-abb6-cf3ff4a30c6d. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb8eaa24-821d-4fb6-acf7-ea01e3b85ad5-0')

In [135]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 0ec58752-0062-4b05-a342-c4256e369cef not found for run c22ad19f-71f3-434d-af05-51696c8358fb. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 171, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c77152bb-847c-49bc-9bb3-8658bd70fe1c-0')

In [136]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 변경하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run a2a143d4-d597-446d-afa2-c12ef92c36cc not found for run 18fb22c3-cb24-4922-be01-afa48e74c540. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 190, 'total_tokens': 192}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e06e6ec6-d64d-480a-ab79-b793f847de0f-0')

## 실험 체인 생성

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "order_id:{order_id}"),
    ]
)

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [12]:
classify_chain = order_change_cancel_prompt | model

In [13]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_chain_with_memory = RunnableWithMessageHistory(
    classify_chain,
    get_session_history,
    input_messages_key="order_id",
    history_messages_key="chat_history",
)

In [15]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 69269b3c-174f-49f6-aefd-d949896c37cd not found for run dde42eb9-d775-49bf-a2e0-76fa1eedffef. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 83, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79530680-f4ca-4ccf-9208-2908dd63632c-0')

In [18]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0')]

In [20]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 2eb3a18c-91d1-4301-8ae2-b6c28f6083c5 not found for run 00408ca8-c8f7-4591-9a8e-926bf6581083. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 102, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec19f61f-1e55-46c6-88f5-6b057bfe8f6c-0')

classify_chain_with_memory의 출력은 저장이 안 되네?

In [21]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0'),
 HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 169, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0b352454-e795-485a-a26a-c38bada679c7-0')]

## order_id 주어졌을 때 처리 체인

In [25]:
response = classify_chain_with_memory.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run a5270db7-107f-4bab-a48c-3506bc7e2053 not found for run 9262573e-ab0b-4e2f-bd27-3d2c314a8eed. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


In [35]:
def fetch_recent_orders(dict):
    print(dict)
    return "produdct: 백설기 quantity: 2 price: 13,000"

In [41]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the check_purpose and queried_result.
            create a message to show the queried_result and ask for final confirmation considering check_purpose.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "check_purpose:{check_purpose}\nqueried_result:{queried_result}")
    ]
)

In [42]:
generate_confirm_message_chain = generate_confirm_message_prompt | model

In [43]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

test_chain1 = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | generate_confirm_message_chain
test_chain1

{
  order_id: RunnablePassthrough(),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from 

In [44]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddadc334-cda2-4ae0-83bf-40cc0b23938a-0'),
 HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 169, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0b352454-e795-485a-a26a-c38bada679c7-0')]

In [45]:
test_chain1.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 28438fb1-5617-465c-84f5-46f8bb2e1502 not found for run 4280a95f-4268-45d5-be10-30dff4d1c810. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 내용을 확인해보실래요?\n\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n주문 변경을 원하시나요? 최종 확인 부탁드립니다.', response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 185, 'total_tokens': 251}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-215495d7-2c89-4565-8cdd-cdb7b4a3bbf0-0')

---

In [46]:
def debug(dict):
    return dict

In [47]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

try_fix_chain = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | RunnableLambda(debug)
try_fix_chain

{
  order_id: RunnablePassthrough(),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from 

In [48]:
try_fix_chain.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 47b42aa4-32f2-4813-8301-ba207c062577 not found for run c8fac5b0-f8c0-4253-8c93-9cfe33b3900b. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


{'order_id': {'order_id': 3},
 'check_purpose': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 102, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4c373960-6f10-4750-b92c-a178700cf3d1-0'),
 'queried_result': 'produdct: 백설기 quantity: 2 price: 13,000'}

In [50]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()

In [51]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

try_fix_chain = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory | str_parser,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | RunnableLambda(debug)
try_fix_chain

{
  order_id: RunnablePassthrough(),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from 

In [52]:
try_fix_chain.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 1a4e0759-d493-482a-b938-614b5d870405 not found for run b503f226-03e9-4427-8cb0-c5268147188f. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


{'order_id': {'order_id': 3},
 'check_purpose': '주문 변경',
 'queried_result': 'produdct: 백설기 quantity: 2 price: 13,000'}

In [53]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

try_fix_chain = {
    "order_id": lambda input: input["order_id"],
    "check_purpose": classify_chain_with_memory | str_parser,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | RunnableLambda(debug)
try_fix_chain

{
  order_id: RunnableLambda(...),
  check_purpose: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
                   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
                 }), config={'run_name': 'insert_history'})
                 | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from th

In [54]:
try_fix_chain.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 0e9721e1-48b9-4369-9a0b-439b8f126cde not found for run 5af93389-9543-4111-b90c-d1f3b21dee76. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


{'order_id': 3,
 'check_purpose': '주문 변경',
 'queried_result': 'produdct: 백설기 quantity: 2 price: 13,000'}

---

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

test_chain1 = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | generate_confirm_message_chain
test_chain1

In [56]:
test_chain2 = (
    classify_chain_with_memory | 
    RunnablePassthrough.assign(check_purpose=classify_chain_with_memory) | 
    RunnablePassthrough.assign(queried_result=RunnableLambda(fetch_recent_orders)) |
    generate_confirm_message_chain
    )
test_chain2 

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(vari

In [57]:
test_chain2.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 94f73d1e-cd00-49dd-bce4-fb2967cf08bc not found for run 3bbec3cf-a86f-4ed7-bdc7-ba980409a60e. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AssertionError: The input to RunnablePassthrough.assign() must be a dict.

## 확인 요청 메시지 작성 이후 부분

In [ ]:
test_chain1.invoke(
    {"order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )

Parent run 28438fb1-5617-465c-84f5-46f8bb2e1502 not found for run 4280a95f-4268-45d5-be10-30dff4d1c810. Treating as a root run.


{'order_id': 3}


Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got 3.')


AIMessage(content='주문 내용을 확인해보실래요?\n\n상품: 백설기\n수량: 2개\n가격: 13,000원\n\n주문 변경을 원하시나요? 최종 확인 부탁드립니다.', response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 185, 'total_tokens': 251}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-215495d7-2c89-4565-8cdd-cdb7b4a3bbf0-0')

# 요청 확인 부분 

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

test_chain1 = {
    "order_id": RunnablePassthrough(),
    "check_purpose": classify_chain_with_memory,
    "queried_result": RunnableLambda(fetch_recent_orders)
} | generate_confirm_message_chain
test_chain1

## memory 공유 확인 위한 dummy chain 생성

In [61]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [74]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [75]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [76]:
dummy_chain = dummy_prompt | model 

In [77]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

ValidationError: 1 validation error for RunnableWithMessageHistory
input_messages_key
  str type expected (type=type_error.str)

In [66]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 710052a1-b37d-455c-bbe6-687714c6eafe not found for run f0cb69af-44d7-4ae9-b080-8c74ce697107. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b68da99e-cdad-4e22-ae65-06f52fc132fc-0')

In [68]:
response = dumy_chain_with_memory.invoke(
    {"input": None, 
    "order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 0a619675-f802-47b4-9663-56b4d0b6222a not found for run 2006defa-a6cf-45a6-a7fe-567c37196141. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


AIMessage(content='조회', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 162, 'total_tokens': 164}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bd1c1362-266f-47f9-88aa-2379272c82fb-0')

In [70]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b68da99e-cdad-4e22-ae65-06f52fc132fc-0')]

## 프롬프트 수정

In [94]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types.
            You need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            After classifying into '주문 변경' or '주문 취소', you need to determine whether the user has agreed to the action (yes or no) based on the recent conversation and user input.
            Respond with a JSON object in the following format:
            {"check_purpose": "<주문 변경 or 주문 취소>", "confirmation": "<yes or no>"}
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input: {input}\norder_id:{order_id}"),
    ]
)
print(order_change_cancel_prompt)

input_variables=['"check_purpose"', 'chat_history', 'input', 'order_id'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['"check_purpose"'], template='\n            You are a robot that classifies customer input messages into specific types.\n            You need to review the conversation from the latest to the oldest and output either \'주문 변경\' or \'주문 취소\'.\n            After classifying into \'주문 변경\' or \'주문 취소\', you need to determine whether the user has agreed to the action (yes or no) based on the recent conversation and user input.\n            Respond with a JSON object in the following format:\n            {"check_purpose": "<주문 변경 o

In [102]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types.
            You need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            After classifying into '주문 변경' or '주문 취소', you need to determine whether the user has agreed to the action (yes or no) based on the recent conversation and user input.
            Respond with a JSON object in the following format:
            "check_purpose": "<주문 변경 or 주문 취소>", "confirmation": "<yes or no>"
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "order_id:{order_id}\ninput:{input}")
    ]
)


In [103]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [104]:
classify_chain = order_change_cancel_prompt | model
classify_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that classifies customer input messages into specific types.\n            You need to review the conversation from the latest to the oldest and output either \'주문 변경\' or \'주문 취소\'.\n            After classifying into \'주문 변경\' or \'주문 취소\', you need to determine whether the user has agreed to the action (yes or no) based on the recent conversation and user input.\n            Respond with a JSON object in the following format:\n            "check_purpose": "<주문 변경 or 주문 취소>", "conf

In [105]:
from langchain_core.runnables.history import RunnableWithMessageHistory

test_chain3 = RunnableWithMessageHistory(
    classify_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [106]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b68da99e-cdad-4e22-ae65-06f52fc132fc-0')]

In [107]:
response = test_chain3.invoke(
    {"input": None, 
    "order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 0d19f63c-4613-4cf3-ac37-2af962ec012d not found for run 50dfb1fe-0f57-4279-bfaf-203c75b519c7. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


AIMessage(content='주문 취소, 확인하시겠습니까?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 166, 'total_tokens': 182}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-96fa79c7-e88f-4e79-ade0-5f65aea2bc1d-0')

# 파서 사용

In [123]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Define your desired data structure.
class OrderChangeCancelConfirmation(BaseModel):
    check_purpose: str = Field(description="주문 변경 or 주문 취소")
    confirmation: str = Field(description="yes or no")

# Set up a parser.
parser = JsonOutputParser(pydantic_object=OrderChangeCancelConfirmation)

# Create the prompt.
order_change_cancel_prompt = PromptTemplate(
    template=("""
        You are a robot that classifies customer input messages into specific types.
        You need to review the conversation from the latest to the oldest and output either '주문 변경' or '주문 취소'.
        
        After classifying into '주문 변경' or '주문 취소', you need to check if there is an AI message showing the order details for cancellation confirmation, and whether the user agreed to this cancellation confirmation message.
        Based on this, determine the 'confirmation' field value as 'yes' if both conditions are satisfied.
        
        chat_history: {chat_history}
        customer_input: {input}
        order_id: {order_id}
        
        chat_history: {chat_history}
        customer_input: {input}
        order_id: {order_id}
        
        {format_instructions}
    """),
    input_variables=["chat_history", "order_id", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

order_change_cancel_prompt


PromptTemplate(input_variables=['chat_history', 'input', 'order_id'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"check_purpose": {"description": "\\uc8fc\\ubb38 \\ubcc0\\uacbd or \\uc8fc\\ubb38 \\ucde8\\uc18c", "title": "Check Purpose", "type": "string"}, "confirmation": {"description": "yes or no", "title": "Confirmation", "type": "string"}}, "required": ["check_purpose", "confirmation"]}\n```'}, template="\n        You are a robot that classifies customer input messages into specific types.\n        You need to review

In [124]:
model = ChatOpenAI(temperature=0)

In [125]:
classify_chain = order_change_cancel_prompt | model | parser
classify_chain

PromptTemplate(input_variables=['chat_history', 'input', 'order_id'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"check_purpose": {"description": "\\uc8fc\\ubb38 \\ubcc0\\uacbd or \\uc8fc\\ubb38 \\ucde8\\uc18c", "title": "Check Purpose", "type": "string"}, "confirmation": {"description": "yes or no", "title": "Confirmation", "type": "string"}}, "required": ["check_purpose", "confirmation"]}\n```'}, template="\n        You are a robot that classifies customer input messages into specific types.\n        You need to review

In [126]:
from langchain_core.runnables.history import RunnableWithMessageHistory

test_chain3 = RunnableWithMessageHistory(
    classify_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [127]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b68da99e-cdad-4e22-ae65-06f52fc132fc-0')]

In [128]:
response = test_chain3.invoke(
    {"input": None, 
    "order_id": 3},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run 5fa2fdf9-9058-4ae1-8f7c-ae8d406eeeb5 not found for run cc2b9434-13d3-494c-bb13-394da1b91dbb. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


{'check_purpose': '주문 취소', 'confirmation': 'yes'}

In [122]:
print(response)

{'check_purpose': '주문 취소', 'confirmation': 'yes'}
